In [ ]:
import pandas as pd
df = pd.read_csv('/content/GSE64881_segmentation_at_30000bp.passqc.multibam.csv')
feature_df = pd.read_csv('/content/Hist1_region_features.csv')
df

,chrom,start,stop,F10A2,F10A3,F10A4,F10A5,F10A6,F10B1,F10B2,...,F9F2,F9F3,F9F4,F9F5,F9F6,F9G1,F9G2,F9G3,F9G4,F9G5
0,chr1,0,30000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,chr1,30000,60000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,chr1,60000,90000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,chr1,90000,120000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,chr1,120000,150000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90872,chr4_random,150000,160594,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
90873,chr3_random,0,30000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
90874,chr3_random,30000,41899,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
90875,chrM,0,16299,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


Extracting Hist1 region

Data frame 2 (df2) has hist1 region of mouse genome

In [ ]:
df2 = df[(df['chrom']=='chr13') & (df['start']>= 21690000) & (df['stop']<= 24120000)]

Reading the feature table

In [ ]:
hist_data = df2.loc[:, (df2 != 0).any(axis=0)]
hist_data = hist_data.drop(["chrom","start","stop"], axis=1)
hist_data

,F10A3,F10A5,F10B3,F10B5,F10C1,F10C3,F10C4,F10D1,F10D5,F10E3,...,F9A4,F9A5,F9B4,F9B6,F9C5,F9D3,F9D6,F9F2,F9F3,F9G3
69714,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
69715,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1
69716,1,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,1
69717,0,0,0,1,0,1,0,1,0,0,...,0,0,0,1,1,0,0,1,0,0
69718,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69790,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,1,1,0,0,0
69791,0,0,0,1,0,1,0,1,0,0,...,0,0,0,0,0,1,1,1,0,0
69792,0,0,0,1,0,1,0,1,0,0,...,0,0,0,0,1,1,1,1,0,0
69793,1,0,0,1,0,1,0,1,0,0,...,0,1,0,0,1,1,1,1,0,0


Determining if there is any pattern of below features in each cluster and removing the others

1. Histone genes
2. Vmn genes
3. Lamina associated domains
4. RNAPII-S2P
5. RNAPII-S5P
6. RNAPII-S7P
7. enhancer
8. H3K9me3
9. H3K20me3
10. H3K27me3
11. H3K36me3
12. NANOG
13. pou5f1
14. sox2
15. CTCF


In [ ]:
feature_data = feature_df.drop(["name","CHD2","EP300", "ZNF384", "ZC3H11A", "HCFC1", "MAFK", "med12", "tcf3"], axis=1)
feature_data_list = list(feature_data.columns)

In [ ]:
feature_data_list
# print(len(feature_data_list))

['CTCF-7BWU',
 'NANOG',
 'LAD',
 'RNAPII-S5P',
 'RNAPII-S7P',
 'h3k27me3',
 'Hist1',
 'Vmn',
 'H3K9me3',
 'pou5f1',
 'sox2',
 'H3K36me3',
 'RNAPII-S2P',
 'Enhancer',
 'H3K20me3']

In [ ]:
count_of_windows = hist_data.sum(axis=0)
column_names = list(hist_data.columns)

# **Clustering**

In [ ]:
import numpy as np

rows = 163
cols = 163


m_matrix = np.zeros((rows, cols))

for x in range(rows):

  for y in range(x+1, cols):

    count_m11=0

    for z in range(81):

      if ((hist_data.iat[z,x] == 1) and (hist_data.iat[z,y] == 1)):
        count_m11 += 1;

    m_matrix[x][y] = ((count_m11)/min(count_of_windows[x],count_of_windows[y]))

In [ ]:
for i in range(rows):
    for j in range(i, cols):
        m_matrix[j][i] = m_matrix[i][j]
print(np.matrix(m_matrix))

[[0.         1.         0.         ... 0.52631579 0.53846154 0.8       ]
 [1.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.52631579 0.         0.         ... 0.         0.38461538 0.33333333]
 [0.53846154 1.         0.         ... 0.38461538 0.         0.4       ]
 [0.8        0.         0.         ... 0.33333333 0.4        0.        ]]


In [ ]:
import random
numbers = [0,1,2]
store_variance = float('inf')
store_medoids = []
store_cluster_1 = []
store_cluster_2 = []
store_cluster_3 = []


for item1 in range(40):
  K=3
  medoids = list(np.random.choice(column_names,K))

  for item2 in range(40):
    dict_cluster = {medoids[0]:[],medoids[1]:[],medoids[2]:[]}
    medoid_index_1 = column_names.index(medoids[0])
    medoid_index_2 = column_names.index(medoids[1])
    medoid_index_3 = column_names.index(medoids[2])

    for item3 in column_names:
      if item3 not in medoids:

        item_ind = column_names.index(item3)

        if(m_matrix[item_ind][medoid_index_1] > m_matrix[item_ind][medoid_index_2] and m_matrix[item_ind][medoid_index_1] > m_matrix[item_ind][medoid_index_3]):
          dict_cluster[medoids[0]].append(item3)

        elif(m_matrix[item_ind][medoid_index_2] > m_matrix[item_ind][medoid_index_1] and m_matrix[item_ind][medoid_index_2] > m_matrix[item_ind][medoid_index_3]):
          dict_cluster[medoids[1]].append(item3)

        elif(m_matrix[item_ind][medoid_index_3] > m_matrix[item_ind][medoid_index_1] and m_matrix[item_ind][medoid_index_3] > m_matrix[item_ind][medoid_index_2]):
          dict_cluster[medoids[2]].append(item3)

        elif(m_matrix[item_ind][medoid_index_3] == m_matrix[item_ind][medoid_index_1] and m_matrix[item_ind][medoid_index_3] == m_matrix[item_ind][medoid_index_2]):
          dict_cluster[medoids[random.choice(numbers)]].append(item3)

        elif(m_matrix[item_ind][medoid_index_3] == m_matrix[item_ind][medoid_index_2]):
          dict_cluster[medoids[random.choice([2,1])]].append(item3)

        elif(m_matrix[item_ind][medoid_index_1] == m_matrix[item_ind][medoid_index_2]):
          dict_cluster[medoids[random.choice([0,1])]].append(item3)

        else:
          dict_cluster[medoids[random.choice([0,2])]].append(item3)


    keys = list(dict_cluster.keys())
    if (len(keys)==3):

      final_variance = 0
      for key, val in dict_cluster.items():

        sum = 0
        value_1 = column_names.index(key)
        for item_1 in val:
          value_2 = column_names.index(item_1)
          sum += (m_matrix[value_1][value_2])
        if(len(dict_cluster[key])!=0):
          v1_value = (sum/float(len(dict_cluster[key])))
        final_variance = final_variance + v1_value

      if final_variance <= store_variance:
        store_variance = final_variance
        store_centroids = medoids
        store_cluster_1 = dict_cluster[keys[0]]
        store_cluster_2 = dict_cluster[keys[1]]
        store_cluster_3 = dict_cluster[keys[2]]

    cluster_1 = list(dict_cluster[medoids[0]])
    cluster_1.append(medoids[0])

    cluster_2 = list(dict_cluster[medoids[1]])
    cluster_2.append(medoids[1])

    cluster_3 = list(dict_cluster[medoids[2]])
    cluster_3.append(medoids[2])

    def find_max_distance_index(lst):
        max_distance_sum = float('-inf')
        max_distance_index = -1
        for i in range(len(lst)):
            distance_sum = 0
            for j in range(len(lst)):
                if i != j:
                    distance_sum += m_matrix[i][j]
            if distance_sum >= max_distance_sum:
                max_distance_sum = distance_sum
                max_distance_index = i
        return max_distance_index

    medoids[0] = cluster_1[find_max_distance_index(cluster_1)]
    medoids[1] = cluster_2[find_max_distance_index(cluster_2)]
    medoids[2] = cluster_3[find_max_distance_index(cluster_3)]


In [ ]:
print(store_centroids)
print(store_variance)
print(store_cluster_1)
print(store_cluster_2)
print(store_cluster_3)
print(len(store_cluster_1))
print(len(store_cluster_2))
print(len(store_cluster_3))

['F10D1', 'F15E2', 'F10F3']
0.9808752927400468
['F10A3', 'F10B3', 'F10B5', 'F10C4', 'F10D1', 'F10E3', 'F10E5', 'F10F1', 'F10G2', 'F11A3', 'F11A6', 'F11B2', 'F11D1', 'F11D2', 'F11D4', 'F12B1', 'F12B2', 'F12E1', 'F15A3', 'F15D1', 'F15D2', 'F15E2', 'F15F3', 'F15F6', 'F15H1', 'F15H5', 'F16A2', 'F16A5', 'F16B6', 'F17B1', 'F17C1', 'F17C7', 'F17D6', 'F17E6', 'F17F5', 'F17G3', 'F17G5', 'F18B4', 'F18E1', 'F18F2', 'F3F1', 'F5A2', 'F5A4', 'F5B4', 'F5E2', 'F5G2', 'F6E2', 'F6E4', 'F6F2', 'F7B1', 'F7B2', 'F7B4', 'F7B5', 'F7C1', 'F7C2', 'F7D6', 'F7G2', 'F8B5', 'F8E6', 'F9D6', 'F9F2']
['F10A5', 'F11B1', 'F11G3', 'F12C6', 'F12D4', 'F12E4', 'F12E6', 'F12F5', 'F15A5', 'F15A6', 'F15C6', 'F15C8', 'F15E1', 'F16A4', 'F16B2', 'F16B3', 'F16C1', 'F16D6', 'F16G6', 'F17A7', 'F17B7', 'F17C2', 'F17E7', 'F17F3', 'F18B2', 'F18D2', 'F18E2', 'F5B1', 'F5B2', 'F5B3', 'F5C1', 'F5C2', 'F5C4', 'F5D1', 'F5F2', 'F6A4', 'F6B2', 'F6B4', 'F6C4', 'F6G2', 'F6G3', 'F7B6', 'F7D4', 'F7E6', 'F7F6', 'F8A1', 'F8A2', 'F8B2', 'F8C5', 'F8D

In [ ]:
store_cluster_1.append(store_centroids[0])
store_cluster_2.append(store_centroids[1])
store_cluster_3.append(store_centroids[2])
df_cluster_1 = hist_data[store_cluster_1]
df_cluster_2 = hist_data[store_cluster_2]
df_cluster_3 = hist_data[store_cluster_3]
matrix_cluster_1 = df_cluster_1.to_numpy()
matrix_cluster_2 = df_cluster_2.to_numpy()
matrix_cluster_3 = df_cluster_3.to_numpy()

For each NP in the cluster, calculate percentage of windows in the NP that contain each of the above mentioned features and calculate average percentage of windows that contain the feature.

In [ ]:
feature_cluster1_percentage_list = []

for feature1 in feature_data_list:
    feature_list1 = list(feature_data[feature1])
    feature_percentage1 = []
    for np1 in store_cluster_1:
      np_list1 = list(hist_data[np1])
      feature_m11_cluster1 = 0

      for value1 in range(len(hist_data[np1])):
        if(feature_list1[value1]==1 and np_list1[value1]==1):
          feature_m11_cluster1  = feature_m11_cluster1 + 1

      percentage1 = ((feature_m11_cluster1)/hist_data[np1].sum())*100
      feature_percentage1.append(percentage1)
    feature_cluster1_percentage_list.append(feature_percentage1)


In [ ]:

import pandas as pd
feature_data_frame_cluster1 = pd.DataFrame(feature_cluster1_percentage_list).transpose()
feature_data_frame_cluster1.columns = feature_data_list

average_cluster1 = list(feature_data_frame_cluster1.mean(axis=0))


In [ ]:
average_cluster1

[22.47309110079033,
 11.685893099332997,
 39.683665040143524,
 59.251293359481004,
 52.09047424999489,
 11.21724773038145,
 25.989098186392898,
 24.385997862768722,
 11.247937097298609,
 23.97112347468198,
 21.12277418257939,
 19.36333571689565,
 35.950857195700586,
 24.520496116134066,
 20.303650329994547]

Visualizing with Radar chart

In [ ]:
import plotly.express as px
import pandas as pd
df = pd.DataFrame(dict(
    r=average_cluster1,
    theta=feature_data_list))
fig = px.line_polar(df, r='r', theta='theta', line_close=True)
fig.show()

In [ ]:
feature_cluster2_percentage_list = []

for feature2 in feature_data_list:
    feature_list2 = list(feature_data[feature2])
    feature_percentage2 = []
    for np2 in store_cluster_2:
      np_list2 = list(hist_data[np2])
      feature_m11_cluster2 = 0

      for value2 in range(len(hist_data[np2])):
        if(feature_list2[value2]==1 and np_list2[value2]==1):
          feature_m11_cluster2  = feature_m11_cluster2 + 1

      percentage2 = ((feature_m11_cluster2)/hist_data[np2].sum())*100
      feature_percentage2.append(percentage2)
    feature_cluster2_percentage_list.append(feature_percentage2)

In [ ]:
import pandas as pd
feature_data_frame_cluster2 = pd.DataFrame(feature_cluster2_percentage_list).transpose()
feature_data_frame_cluster2.columns = feature_data_list

average_cluster2 = list(feature_data_frame_cluster2.mean(axis=0))

In [ ]:
average_cluster2

[18.438687352807598,
 20.341222402178715,
 49.024119426075586,
 47.49188527066747,
 40.779184121068035,
 10.520265397975336,
 18.97889031221223,
 35.236638152876885,
 8.343504663874908,
 24.162422647018467,
 22.89210096502935,
 15.993802098230884,
 28.429192906533633,
 24.711663959437544,
 17.58820944624813]

Visualizing with Radar chart

In [ ]:
import plotly.express as px
import pandas as pd
df = pd.DataFrame(dict(
    r=average_cluster2,
    theta=feature_data_list))
fig = px.line_polar(df, r='r', theta='theta', line_close=True)
fig.show()

Visualizing with Radar chart

In [ ]:
feature_cluster3_percentage_list = []

for feature3 in feature_data_list:
    feature_list3 = list(feature_data[feature3])
    feature_percentage3 = []
    for np3 in store_cluster_3:
      np_list3 = list(hist_data[np3])
      feature_m11_cluster3 = 0

      for value3 in range(len(hist_data[np3])):
        if(feature_list3[value3]==1 and np_list3[value3]==1):
          feature_m11_cluster3  = feature_m11_cluster3 + 1

      percentage3 = ((feature_m11_cluster3)/hist_data[np3].sum())*100
      feature_percentage3.append(percentage3)
    feature_cluster3_percentage_list.append(feature_percentage3)

In [ ]:
import pandas as pd
feature_data_frame_cluster3 = pd.DataFrame(feature_cluster3_percentage_list).transpose()
feature_data_frame_cluster3.columns = feature_data_list

average_cluster3 = list(feature_data_frame_cluster3.mean(axis=0))

In [ ]:
average_cluster3

[17.625304394511673,
 17.36736640350131,
 26.191879211060805,
 55.43914205389432,
 51.24253329960982,
 17.32859746898815,
 19.05552631492967,
 17.96425144206941,
 10.626683558162489,
 24.490658756049257,
 22.23072608100912,
 24.109601769395486,
 32.34073937354889,
 29.40042500973508,
 16.99444726080534]

Visualizing with Radar chart

In [ ]:
import plotly.express as px
import pandas as pd
df = pd.DataFrame(dict(
    r=average_cluster3,
    theta=feature_data_list))
fig = px.line_polar(df, r='r', theta='theta', line_close=True)
fig.show()